In [1]:
import  warnings
from    selenium import webdriver
from    selenium.webdriver.common.keys import Keys 
from    selenium.webdriver.common.action_chains import ActionChains
from    selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from    bs4      import BeautifulSoup
import  time
import  pyperclip
import  requests
import  datetime
import  pymssql
import  pandas as pd
import  chromedriver_autoinstaller
import  subprocess
from    selenium import webdriver
from    selenium.webdriver.chrome.options import Options
from    selenium.webdriver.common.alert import Alert
import  chromedriver_autoinstaller
import  subprocess
import  shutil 
import  xlrd
import  openpyxl

warnings.filterwarnings('ignore')

conn = pymssql.connect(server='localhost', user='pyuser', password='Dnflskfk12!', database='mytest' , charset='utf8') 

db_processing_query = '''
declare @date date = convert(char(10), getdate(), 120)

insert into [mytest].dbo.[selling_zeny_statistics]
select      convert(char(10), dateadd(dd, -1, @date), 120)
            ,aaaa.server
            ,coalesce(bbbb.trade_cnt, 0) as trade_cnt
            ,coalesce(bbbb.selling_zeny_amount, 0) as selling_zeny_amount
            ,coalesce(bbbb.cash_amount, 0) as cash_amount
            ,coalesce(bbbb.one_billion_zeny, 0) as one_billion_zeny
from        svr_idx  aaaa
left join
(
    select  aaa.regdate
            ,aaa.server
            ,count(1) as 'trade_cnt'
            ,sum(aaa.zeny) as 'selling_zeny_amount'
            ,sum(aaa.cash) as 'cash_amount'
            ,100000000 / (sum(aaa.zeny)/sum(aaa.cash))  as 'one_billion_zeny'
    from 
    (
        select  game
                ,server
                ,is_trade_finished
                ,zeny_sell_title
                ,convert(bigint, billion * 100000000.0 + million) as zeny
                ,cash
                ,convert(char(10), trim(regdate), 120) as regdate
        from 
            (
            select  trim(game) as game
                    ,trim(server) as server
                    ,trim(is_trade_finished) as is_trade_finished
                    ,zeny_sell_title 
                    ,cash
                    ,case
                        when CHARINDEX('억', zeny, 1) > 0
                            then replace(left(zeny, CHARINDEX('억', zeny,1)), '억', '') else 0 end 
                    as billion
                    ,case
                        when CHARINDEX('만', zeny, 1) > 0
                            then  convert(bigint, replace(right(zeny, 5), '만', '') *10000 ) else 0 end  
                    as million
                    ,case 
                        when regdate like N'%전%' then convert(char(10), dateadd(dd, -1, @date), 120) 
                        when regdate like N'%/%'  then convert(char(10), '2021'+'-'+ replace(regdate, '/', '-'), 120) 
                        else regdate
                    end 
                    as regdate
            from    mytest..zeny_selling_monitoring with (nolock)
            ) as aa 
    ) as aaa 
    where
            aaa.regdate = convert(char(10), dateadd(dd, -1, @date), 120)
    group BY
            aaa.regdate,aaa.server 
) as bbbb
on      aaaa.server=bbbb.server

select * from mytest..zeny_selling_monitoring with (nolock) ; 
'''

data5 = pd.read_sql(sql=db_processing_query, con=conn)

conn.commit()
conn.close()